Playwright API test

In [ ]:
!apt install chromium-chromedriver

!pip install nest_asyncio
!pip install playwright

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 44 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [ ]:
!playwright install

161.3 MiB [] 0% 10.7s161.3 MiB [] 0% 51.9s161.3 MiB [] 0% 35.3s161.3 MiB [] 0% 22.6s161.3 MiB [] 0% 15.4s161.3 MiB [] 0% 14.1s161.3 MiB [] 0% 12.3s161.3 MiB [] 1% 11.5s161.3 MiB [] 1% 10.7s161.3 MiB [] 1% 10.4s161.3 MiB [] 1% 9.5s161.3 MiB [] 2% 9.0s161.3 MiB [] 2% 8.4s161.3 MiB [] 2% 8.3s161.3 MiB [] 3% 7.8s161.3 MiB [] 3% 7.4s161.3 MiB [] 3% 7.3s161.3 MiB [] 4% 7.1s161.3 MiB [] 4% 7.3s161.3 MiB [] 4% 7.1s161.3 MiB [] 4% 6.9s161.3 MiB [] 5% 6.9s161.3 MiB [] 5% 7.1s161.3 MiB [] 6% 7.1s161.3 MiB [] 6% 7.2s161.3 MiB [] 6% 7.1s161.3 MiB [] 6% 7.0s161.3 MiB [] 7% 6.9s161.3 MiB [] 7% 6.8s161.3 MiB [] 8% 6.7s161.3 MiB [] 8% 6.6s161.3 MiB [] 8% 6.5s161.3 MiB [] 9% 6.4s161.3 MiB [] 9% 6.5s161.3 MiB [] 10% 6.4s161.3 MiB [] 10% 6.3s161.3 MiB [] 10% 6.2s161.3 MiB [] 11% 6.1s161.3 MiB [] 11% 6.0s161.3 MiB [] 12% 5.9s161.3 MiB [] 12% 5.8s161.3 MiB [] 13% 5.8s161.3 MiB [] 13% 5.7s161.3 MiB [] 13% 5.8s161.3 MiB [] 14% 5.8s161.3 MiB [] 14% 5.9s161.3 MiB [] 14% 5.7s161.3 MiB [] 15% 5.6s161.3 MiB [] 15%

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import pandas as pd
# Apply nest_asyncio to allow running in environments with an existing event loop
nest_asyncio.apply()

async def check_accessibility(url):
    try:
        async with async_playwright() as p:
            # Start Chromium browser
            browser = await p.chromium.launch()
            page = await browser.new_page()

            # Navigate to the URL
            await page.goto(url)

            # Inject axe-core from CDN
            await page.add_script_tag(url="https://cdn.jsdelivr.net/npm/axe-core@4.4.1/axe.min.js")

            # Run axe accessibility checks
            results = await page.evaluate("""
            () => axe.run(document, {
                runOnly: {
                    type: 'tag',
                    values: ['wcag2a', 'wcag2aa']
                }
            })
            """)

            # Close the browser
            await browser.close()

            # Process and return results
            return results
    except Exception as e:
        print("An error occurred:", e)
        return None

async def main(url):

    url_to_test = url
    print(f"URL to test: {url_to_test}")
    results = await check_accessibility(url_to_test)

    if results:
        # Extract and print violations
        violations = results['violations']
        if violations:
            print(f"Number of accessibility violations: {len(violations)}")
            for violation in violations:
                print(f"\nDescription: {violation['description']}")
                print(f"Impact: {violation['impact']}")
                print(f"Help URL: {violation['helpUrl']}")
                for node in violation['nodes']:
                    print(f"Element affected: {node['html']}")

        else:
            print("No accessibility violations found.")

    else:
        print("No results returned or an error occurred.")


# Running the main function with asyncio
if __name__ == "__main__":
    asyncio.run(main("https://github.com/"))



URL to test: https://github.com/
An error occurred: Page.add_script_tag: Refused to load the script 'https://cdn.jsdelivr.net/npm/axe-core@4.4.1/axe.min.js' because it violates the following Content Security Policy directive: "script-src github.githubassets.com". Note that 'script-src-elem' was not explicitly set, so 'script-src' is used as a fallback.

No results returned or an error occurred.


In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import pandas as pd

# Apply nest_asyncio to allow running in environments with an existing event loop
nest_asyncio.apply()

async def check_accessibility(url):
    try:
        async with async_playwright() as p:
            # Start Chromium browser
            browser = await p.chromium.launch()
            page = await browser.new_page()

            # Navigate to the URL
            await page.goto(url)

            # Inject axe-core from CDN
            await page.add_script_tag(url="https://cdn.jsdelivr.net/npm/axe-core@4.4.1/axe.min.js")

            # Run axe accessibility checks
            results = await page.evaluate("""
            () => axe.run(document, {
                runOnly: {
                    type: 'tag',
                    values: ['wcag2a', 'wcag2aa']
                }
            })
            """)

            # Close the browser
            await browser.close()

            # Process and return results
            return results
    except Exception as e:
        print("An error occurred:", e)
        return None

async def main(url, index, impactScore, df):
    url_to_test = url
    print(f"URL to test: {url_to_test}")
    results = await check_accessibility(url_to_test)

    if results:
        # Extract and print violations
        violations = results['violations']
        if violations:
            print(f"Number of accessibility violations: {len(violations)}")

            for violation in violations:
                description = violation['description']
                impact = violation['impact']
                help_url = violation ['helpUrl']
                html_code = ", ".join([node['html'] for node in violation['nodes']])

                # Add data to the DataFrame
                df.loc[index] = [url, len(violations), violation['id'], impactScore.get(impact, "Unknown"), description, help_url, html_code]
                index += 1  # Increment index for next row if needed

        else:
            print("No accessibility violations found.")
            df.loc[index] = [url, 0, None, 0, "No violations found", None, None]
    else:
        print("No results returned or an error occurred.")

# Prepare DataFrame and impactScore dictionary for the example
columns = ['webURL', 'numViolations', 'violationnumberID', 'initialImpactScore', 'Description', 'LongDescription', 'AffectedHTMLElement(s)']
df = pd.DataFrame(columns=columns)
impactScore = {
  "critical": 5,
  "serious": 4,
  "moderate": 3,
  "minor": 2,
  "cosmetic": 1,
}

# Running the main function with asyncio
if __name__ == "__main__":
    asyncio.run(main("https://els-egypt.net/", 0, impactScore, df))
    print(df)


URL to test: https://els-egypt.net/
Number of accessibility violations: 3
                   webURL  numViolations violationnumberID  \
0  https://els-egypt.net/              3       button-name   
1  https://els-egypt.net/              3    color-contrast   
2  https://els-egypt.net/              3         image-alt   

   initialImpactScore                                        Description  \
0                   5              Ensures buttons have discernible text   
1                   4  Ensures the contrast between foreground and ba...   
2                   5  Ensures <img> elements have alternate text or ...   

                                     LongDescription  \
0  https://dequeuniversity.com/rules/axe/4.4/butt...   
1  https://dequeuniversity.com/rules/axe/4.4/colo...   
2  https://dequeuniversity.com/rules/axe/4.4/imag...   

                              AffectedHTMLElement(s)  
0  <button class="app-header__action-button app-h...  
1  <a class="btn btn-link btn-block b

In [ ]:
df

,webURL,numViolations,violationnumberID,initialImpactScore,Description,LongDescription,AffectedHTMLElement(s)
0,https://els-egypt.net/,3,button-name,5,Ensures buttons have discernible text,https://dequeuniversity.com/rules/axe/4.4/butt...,"<button class=""app-header__action-button app-h..."
1,https://els-egypt.net/,3,color-contrast,4,Ensures the contrast between foreground and ba...,https://dequeuniversity.com/rules/axe/4.4/colo...,"<a class=""btn btn-link btn-block btn-lg p-0 m-..."
2,https://els-egypt.net/,3,image-alt,5,Ensures <img> elements have alternate text or ...,https://dequeuniversity.com/rules/axe/4.4/imag...,"<img class=""app-hero__slogan"" src=""https://els..."


In [ ]:
#W3C Validator API similar to Playwirght
import requests

def validate_html(url):
    api_url = f"https://validator.w3.org/nu/?doc={url}&out=json"
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Example usage
validation_results = validate_html('https://els-egypt.net/')
print(validation_results)


{'url': 'https://els-egypt.net/', 'messages': [{'type': 'info', 'lastLine': 25, 'lastColumn': 61, 'firstColumn': 6, 'message': 'Trailing slash on void elements has no effect and interacts badly with unquoted attribute values.', 'extract': "yle>\n     <meta name='robots' content='max-image-preview:large' />\n<scri", 'hiliteStart': 10, 'hiliteLength': 56}, {'type': 'info', 'lastLine': 26, 'lastColumn': 31, 'firstColumn': 1, 'subType': 'warning', 'message': 'The “type” attribute is unnecessary for JavaScript resources.', 'extract': 'large\' />\n<script type="text/javascript">\n', 'hiliteStart': 10, 'hiliteLength': 31}, {'type': 'info', 'lastLine': 33, 'lastColumn': 55, 'firstColumn': 1, 'subType': 'warning', 'message': 'The “type” attribute for the “style” element is not needed and should be omitted.', 'extract': "</script>\n<style id='wp-emoji-styles-inline-css' type='text/css'>\n\n\timg", 'hiliteStart': 10, 'hiliteLength': 55}, {'type': 'info', 'lastLine': 47, 'lastColumn': 167, 'firstC

In [ ]:
import pandas as pd

# Create a dictionary with the data
data = {
    "webURL": ["https://els-egypt.net/", "https://els-egypt.net/", "https://els-egypt.net/"],
    "numViolations": [3, 3, 3],
    "violationName": ["button-name", "color-contrast", "image-alt"],
    "ImpactScore": [5, 4, 5],
    "Description": [
        "Ensures buttons have discernible text",
        "Ensures the contrast between foreground and background colors meets WCAG standards",
        "Ensures <img> elements have alternate text or a role of none or presentation"
    ],
    "LongDescription": [
        "Form buttons have a descriptive value, buttons must have appropriate, equivalent alternative text",
        "Contrast ratio between the link and the surrounding text is at least 3:1 ",
        "Images, image buttons, and image map hot spots have appropriate, equivalent alternative text."
    ],
    "AffectedHTMLElement(s)": [
        '<button class="app-header__action-button app-header__action-button--primary">',
        '<a class="btn btn-link btn-block btn-lg p-0 m-0 text-uppercase">',
        '<img class="app-hero__slogan" src="https://els-egypt.net/static/img/slogan.png">'
    ]
}

# Create the dataframe
df = pd.DataFrame(data)

# Display the dataframe
df


,webURL,numViolations,violationName,ImpactScore,Description,LongDescription,AffectedHTMLElement(s)
0,https://els-egypt.net/,3,button-name,5,Ensures buttons have discernible text,"Form buttons have a descriptive value, buttons...","<button class=""app-header__action-button app-h..."
1,https://els-egypt.net/,3,color-contrast,4,Ensures the contrast between foreground and ba...,Contrast ratio between the link and the surrou...,"<a class=""btn btn-link btn-block btn-lg p-0 m-..."
2,https://els-egypt.net/,3,image-alt,5,Ensures <img> elements have alternate text or ...,"Images, image buttons, and image map hot spots...","<img class=""app-hero__slogan"" src=""https://els..."
